<a href="https://colab.research.google.com/github/viveknilawar/CustomChatGPT/blob/main/Tech_at_Work_Fine_Tune_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install openai
! pip install pandas
! pip install gradio

In [ ]:
import openai
import pandas as pd
import gradio as gr

In [ ]:
openai.api_key = "YOUR API KEY"

In [ ]:
file_path = '/contents/DRIVE LOCATION/file.csv'
df = pd.read_csv(file_path)

In [ ]:
df.to_csv('/contents/DRIVE LOCATION/file_pandas.csv', index=False)

In [ ]:
!openai tools fine_tunes.prepare_data -f /content/drive/MyDrive/Airline-Tweets/tweets_pandas.csv

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 1007 prompt-completion pairs
- The input file should contain exactly two columns/keys per row. Additional columns/keys present are: ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13']
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 2 duplicated prompt-completion sets. These are rows: [327, 511]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion s

In [ ]:
def create_training_file(file_path):
  file = openai.File.create(
      file=open(file_path, "rb"),
      purpose="fine-tune"
  )
  return file


training_file = create_training_file("/contents/DRIVE LOCATION/file_prepared.jsonl")
print(training_file)

{
  "bytes": 154537,
  "created_at": 1684538295,
  "filename": "file",
  "id": "file-xjSgFivxFlmkwCsWYv2vtWsO",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


In [ ]:
training_file['id']

'file-xjSgFivxFlmkwCsWYv2vtWsO'

In [ ]:
fine_tuned_model = openai.FineTune.create(training_file=training_file['id'],
model ='ada',
n_epochs=4)
print(fine_tuned_model)

{
  "created_at": 1684538664,
  "events": [
    {
      "created_at": 1684538664,
      "level": "info",
      "message": "Created fine-tune: ft-hKKejuxmB0FazBYlRvJd5Ho6",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-hKKejuxmB0FazBYlRvJd5Ho6",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-pWoX9YzaETZVCbyWbebFq8ld",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 154537,
      "created_at": 1684538295,
      "filename": "file",
      "id": "file-xjSgFivxFlmkwCsWYv2vtWsO",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1684538664,
  "validation_files": []
}


In [ ]:
openai.FineTune.list_events(id="ft-YOUR FILE ABOVE") ['data']

[<OpenAIObject fine-tune-event at 0x7f95dfb54c70> JSON: {
   "created_at": 1684538664,
   "level": "info",
   "message": "Created fine-tune: ft-hKKejuxmB0FazBYlRvJd5Ho6",
   "object": "fine-tune-event"
 },
 <OpenAIObject fine-tune-event at 0x7f95dfb56e80> JSON: {
   "created_at": 1684538809,
   "level": "info",
   "message": "Fine-tune costs $0.05",
   "object": "fine-tune-event"
 },
 <OpenAIObject fine-tune-event at 0x7f95dfb57880> JSON: {
   "created_at": 1684538809,
   "level": "info",
   "message": "Fine-tune enqueued. Queue number: 0",
   "object": "fine-tune-event"
 },
 <OpenAIObject fine-tune-event at 0x7f95df9f2980> JSON: {
   "created_at": 1684538812,
   "level": "info",
   "message": "Fine-tune started",
   "object": "fine-tune-event"
 },
 <OpenAIObject fine-tune-event at 0x7f95dfa0cea0> JSON: {
   "created_at": 1684539017,
   "level": "info",
   "message": "Completed epoch 1/4",
   "object": "fine-tune-event"
 },
 <OpenAIObject fine-tune-event at 0x7f95dfa0da80> JSON: {
   "

In [ ]:
prompt_text = "TEST TWEET \n\n###\n\n"
response = openai.Completion.create(
    model="ada:YOUR MODEL",
    prompt=prompt_text,
    max_tokens=5,
    temperature=0
)
print(response['choices'][0]['text'])

 negative
 negative
 negative


In [ ]:
def generate_completion(user_prompt):
    hidden_context = " "
    prompt = hidden_context + user_prompt
    response = openai.Completion.create(
        model="ada:YOUR MODEL",
        prompt=prompt,
        max_tokens=10,
        temperature=0
    )
    return response.choices[0].text.strip()
iface = gr.Interface(fn=generate_completion,
                     inputs=gr.inputs.Textbox(lines=5, placeholder='Enter a tweet...'),
                     outputs='text',
                     title="Twitter Flight Happiness",
                     description="Based on thousands of tweets, determines if the tweet you entered is positive, negative, or nuetral",
                     input_labels="Tweet",
                     output_labels="Sentiment")

iface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
<ipython-input-62-d90c40c5c2a3>:11: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'input_labels': 'Tweet', 'output_labels': 'Sentiment'}
  iface = gr.Interface(fn=generate_completion,


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://41bf53283239bd397e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
